In [ ]:
import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import torch  

from lightglue import LightGlue, SuperPoint
from lightglue.utils import load_image, rbd


In [ ]:
# Set the device to MPS if available, otherwise fallback to CPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

In [ ]:
from moviepy.editor import VideoFileClip
from pathlib import Path

# Function to crop, trim, zoom, and change the resolution of the video
def crop_trim_zoom_change_resolution(input_video, output_video, zoom_ratio, start_time_in_seconds, duration_in_seconds, target_resolution=(1920, 1080)):
    # Load the video
    video = VideoFileClip(input_video)
    
    # Check if the start time is within the video's duration
    print(f"Video duration: {video.duration} seconds")  # Debug: Show video duration
    if start_time_in_seconds >= video.duration:
        raise ValueError("Start time exceeds the video length.")
    
    # Calculate the end time based on the start time and duration
    end_time_in_seconds = start_time_in_seconds + duration_in_seconds
    
    # Trim the video to the specified start time and duration
    if end_time_in_seconds <= video.duration:
        video = video.subclip(start_time_in_seconds, end_time_in_seconds)
    else:
        print("Warning: Requested end time exceeds video length. Using the remaining duration.")
        video = video.subclip(start_time_in_seconds, video.duration)

    # Get original video dimensions and FPS
    original_width, original_height = video.size
    fps = video.fps
    print(f"Original video size: {original_width}x{original_height}")  # Debug: Show original size
    
    # Check if FPS is None, and set a default if necessary
    if fps is None:
        print("Warning: FPS could not be read from the video. Setting default FPS to 30.")
        fps = 30.0  # Set a default FPS value
    else:
        print(f"Original FPS: {fps}")  # Debug: Show FPS
    
    # Calculate the new width and height based on the zoom ratio
    new_width = int(original_width / zoom_ratio)
    new_height = int(original_height / zoom_ratio)
    print(f"Zoom ratio: {zoom_ratio}")  # Debug: Show zoom ratio
    print(f"Cropped video size: {new_width}x{new_height}")  # Debug: Show cropped size
    
    # Calculate the coordinates to crop the video so that the crop is centered
    x1 = (original_width - new_width) // 2
    y1 = (original_height - new_height) // 2
    x2 = x1 + new_width
    y2 = y1 + new_height
    
    # Crop the video and center it
    cropped_video = video.crop(x1=x1, y1=y1, x2=x2, y2=y2)
    
    # Resize the cropped video to the target resolution (e.g., 1920x1080)
    print(f"Target resolution: {target_resolution}")  # Debug: Show target resolution
    resized_video = cropped_video.resize(target_resolution)
    
    # Write the cropped, trimmed, zoomed, and resized video to the output file with the specified FPS
    resized_video.write_videofile(output_video, codec='libx264', fps=fps)

# Path to the input video
input_video = '/home/oussama/Documents/EPFL/PDS_LUTS/Dataset/DJI_0763.MOV'
input_video_path = Path(input_video)
if not input_video_path.exists():
    raise FileNotFoundError(f"Input video not found: {input_video_path}")

# Path to the output video
output_video = 'Dataset/DJI_0763_output_trimmed.mp4'

# Zoom ratio (e.g., 2.0 means the cropped area is 50% of the original size)
zoom_ratio = 1.0

# Starting time (in seconds) to begin the trimming
start_time_in_seconds = 8  # Start the video after the first 120 seconds

# Duration in seconds to keep from the video
duration_in_seconds = 8  # Keep 90 seconds after the start

# Target resolution (1920x1080)
target_resolution = (3840, 2160)

# Crop, trim, zoom, and change the resolution of the video
crop_trim_zoom_change_resolution(
    input_video=str(input_video_path), 
    output_video=output_video, 
    zoom_ratio=zoom_ratio, 
    start_time_in_seconds=start_time_in_seconds, 
    duration_in_seconds=duration_in_seconds, 
    target_resolution=target_resolution
)


In [ ]:
import os
import cv2
import glob
import pandas as pd
import re

general_path = '/home/oussama/Documents/EPFL/PDS_LUTS/'

# Function to delete all existing files in the output folder
def clear_folder(folder):
    files = glob.glob(os.path.join(folder, '*'))
    for f in files:
        try:
            os.remove(f)
        except OSError as e:
            print(f"Error: {f} : {e.strerror}")

# Step 1: Extract frames and save to 'images' folder
def extract_frames(video_path, output_folder, frame_interval=250):
    # Clear existing files in the output folder
    clear_folder(output_folder)
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}.")
        return

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    frame_idx = 0
    saved_frames = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % frame_interval == 0:
            cv2.imwrite(os.path.join(output_folder, f'{str(frame_idx+200).zfill(4)}.jpg'), frame)
            saved_frames += 1
        frame_idx += 1
    cap.release()
    print(f"Extracted {saved_frames} frames from the video.")

def parse_srt_to_dataframe(srt_file):
    # Define the data structure
    data = {
        'FrameCnt': [], 'Start_Time': [], 'End_Time': [], 'DiffTime_ms': [],
        'ISO': [], 'Shutter': [], 'Fnum': [], 'EV': [], 'CT': [], 
        'Color_Mode': [], 'Focal_Length': [], 'Latitude': [], 
        'Longitude': [], 'Altitude': [], 'Image': [], 'Resolution_B': [], 'Resolution_S': [],
        'Corners_B': [], 'Corners_S': [], 'Homography': [], 'Neighbors': [],
    }

    # Read and parse the SRT file
    with open(srt_file, 'r') as file:
        content = file.read()
        blocks = content.split('\n\n')  # Split into blocks by double newlines

        for block in blocks:
            # Match each field within the block
            frame_count = re.search(r'FrameCnt : (\d+)', block)
            diff_time = re.search(r'DiffTime : (\d+)ms', block)
            time_match = re.search(r'(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})', block)
            iso = re.search(r'\[iso : (\d+)\]', block)
            shutter = re.search(r'\[shutter : ([\d/\.]+)\]', block)
            fnum = re.search(r'\[fnum : (\d+)\]', block)
            ev = re.search(r'\[ev : ([\d\.\-]+)\]', block)
            ct = re.search(r'\[ct : (\d+)\]', block)
            color_mode = re.search(r'\[color_md : (.*?)\]', block)
            focal_len = re.search(r'\[focal_len : (\d+)\]', block)
            latitude = re.search(r'\[latitude : ([-\d.]+)\]', block)
            longitude = re.search(r'\[longtitude : ([-\d.]+)\]', block)  # Using provided typo
            altitude = re.search(r'\[altitude: ([-\d.]+)\]', block)

            # Extract time data if available
            if time_match:
                start_time, end_time = time_match.groups()
            else:
                start_time, end_time = None, None

            # Append data to dictionary, handling None values where needed
            data['FrameCnt'].append(int(frame_count.group(1)) if frame_count else None)
            data['Start_Time'].append(start_time)
            data['End_Time'].append(end_time)
            data['DiffTime_ms'].append(int(diff_time.group(1)) if diff_time else None)
            data['ISO'].append(int(iso.group(1)) if iso else None)
            data['Shutter'].append(shutter.group(1) if shutter else None)
            data['Fnum'].append(int(fnum.group(1)) if fnum else None)
            data['EV'].append(float(ev.group(1)) if ev else None)
            data['CT'].append(int(ct.group(1)) if ct else None)
            data['Color_Mode'].append(color_mode.group(1) if color_mode else None)
            data['Focal_Length'].append(int(focal_len.group(1)) if focal_len else None)
            data['Latitude'].append(float(latitude.group(1)) if latitude else None)
            data['Longitude'].append(float(longitude.group(1)) if longitude else None)
            data['Altitude'].append(float(altitude.group(1)) if altitude else None)
            data['Image'].append(f'{general_path}images/{int(frame_count.group(1))-1}.jpg' if frame_count else None)

    max_length = max(len(column) for column in data.values())
    for key, column in data.items():
        if len(column) < max_length:
            column.extend([None] * (max_length - len(column)))

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(data)
    df['FrameCnt'] = df['FrameCnt'].astype('Int64')
    return df

# Example usage: extract every 40th frame and save to 'images' folder
video_file = '/home/oussama/Documents/EPFL/PDS_LUTS/Dataset/DJI_0763_output_trimmed.mp4'  # Replace with your video file path

frame_interval=10

extract_frames(video_file, 'images', frame_interval)

# Define the path to the SRT file and the output CSV file
srt_file = '/home/oussama/Documents/EPFL/PDS_LUTS/Dataset/DJI_0763.SRT'
output_df_csv = '/home/oussama/Documents/EPFL/PDS_LUTS/Dataset/DJI_0763_data.csv'
output_gps_csv = '/home/oussama/Documents/EPFL/PDS_LUTS/Dataset/DJI_0763_data_filtered.csv'

# Parse the SRT file and save to CSV
df = parse_srt_to_dataframe(srt_file)
df.loc[:, 'FrameCnt'] = df['FrameCnt'] -1
df.to_csv(output_df_csv, index=False)

# Select only every 200th frame
gps_df = df[df['FrameCnt'] % frame_interval == 1]

gps_df.loc[:, 'FrameCnt'] = gps_df['FrameCnt'] -1

# Save the filtered DataFrame to CSV
gps_df.to_csv(output_gps_csv, index=False)
print(f"Filtered data saved to {output_gps_csv}")

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import glob

def clear_folder(folder):
    files = glob.glob(os.path.join(folder, '*'))
    for f in files:
        try:
            os.remove(f)
        except OSError as e:
            print(f"Error: {f} : {e.strerror}")

def haversine(lat1, lon1, lat2, lon2):
    R = 6371e3  # Earth radius in meters
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = (np.sin(delta_phi / 2) ** 2
         + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

def select_frames_by_distance(df, distance_threshold):
    selected_frames = [0]  # Always start with frame 0
    last_lat, last_lon = df.loc[0, 'Latitude'], df.loc[0, 'Longitude']

    for i in range(1, len(df)):
        lat, lon = df.loc[i, 'Latitude'], df.loc[i, 'Longitude']
        dist = haversine(last_lat, last_lon, lat, lon)
        if dist >= distance_threshold:
            selected_frames.append(i)
            last_lat, last_lon = lat, lon
    return selected_frames

def is_not_blurry(frame, threshold=200.0):
    """
    Determine if an image is blurry using the Laplacian variance method.
    If the variance of the Laplacian is below the threshold, consider it blurry.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lap = cv2.Laplacian(gray, cv2.CV_64F)
    variance = lap.var()
    return variance > threshold

def extract_frames_with_distance(video_path, output_folder, csv_file, distance_threshold=50, blur_threshold=200.0):
    # Clear the output folder
    clear_folder(output_folder)
    os.makedirs(output_folder, exist_ok=True)

    # Read the CSV and get the selected frames
    df = pd.read_csv(csv_file)
    selected_frames = select_frames_by_distance(df, distance_threshold)
    
    # Convert to a queue-like structure for processing
    # We'll move through this list as we find suitable frames
    target_indices = selected_frames.copy()
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}.")
        return

    saved_frames = 0
    final_selected_frames = []
    
    current_frame_index = 0
    target_idx_pos = 0  # Position in the target_indices list

    while True:
        ret, frame = cap.read()
        if not ret:
            # No more frames
            break

        # If we've processed all target frames, stop
        if target_idx_pos >= len(target_indices):
            break

        target_frame = target_indices[target_idx_pos]

        if current_frame_index < target_frame:
            # We haven't reached the target frame yet, just continue reading
            current_frame_index += 1
            continue

        if current_frame_index == target_frame:
            # Check if the frame at target_frame is blurry
            if is_not_blurry(frame, blur_threshold):
                # Not blurry, save it
                cv2.imwrite(os.path.join(output_folder, f'{str(current_frame_index).zfill(4)}.jpg'), frame)
                saved_frames += 1
                final_selected_frames.append(current_frame_index)
                target_idx_pos += 1
            else:
                # It's blurry, we need to keep reading subsequent frames until we find a non-blurry one
                # Do not advance target_idx_pos yet, because we still need a suitable frame for this target
                # Just continue the loop; the next iteration will read the next frame and check again
                pass

        else:
            # current_frame_index > target_frame
            # This means we already passed the target frame (likely because it was blurry)
            # and we're checking subsequent frames to find a non-blurry one.
            if is_not_blurry(frame, blur_threshold):
                # Found a suitable frame after the target frame
                cv2.imwrite(os.path.join(output_folder, f'{str(current_frame_index).zfill(4)}.jpg'), frame)
                saved_frames += 1
                final_selected_frames.append(current_frame_index)
                target_idx_pos += 1
            else:
                # Still blurry, keep reading next frames
                pass

        current_frame_index += 1

    cap.release()
    print(f"Extracted {saved_frames} frames from the video.")
    print(f"Selected frame indices (after checking blur): {final_selected_frames}")

# Example usage:
video_file = '/home/oussama/Documents/EPFL/PDS_LUTS/Dataset/DJI_0763.MOV'
output_df_csv = '/home/oussama/Documents/EPFL/PDS_LUTS/Dataset/DJI_0763_data.csv'
extract_frames_with_distance(video_file, 'images_distance', output_df_csv, distance_threshold=60, blur_threshold=200.0)


Extracted 33 frames from the video.
Selected frame indices (after checking blur): [0, 223, 413, 601, 791, 981, 1168, 1358, 1548, 1736, 1923, 2113, 2453, 2628, 2818, 3008, 3373, 3553, 3743, 3931, 4118, 4308, 4498, 4761, 4951, 5141, 5331, 5521, 6073, 6263, 6453, 6643, 6833]


In [ ]:
import numpy as np

# Haversine formula to calculate distance between two GPS coordinates in meters
def haversine(lat1, lon1, lat2, lon2):
    R = 6371e3  # Earth radius in meters
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Get closest neighbors for each image based on GPS data
def get_closest_neighbors(df, max_neighbors=5, max_distance=100):  # max_distance in meters
    neighbors = {}
    num_images = len(df)
    
    for i in range(num_images):
        lat1, lon1 = df.iloc[i]['Latitude'], df.iloc[i]['Longitude']
        distances = []
        
        for j in range(num_images):
            if i != j:
                lat2, lon2 = df.iloc[j]['Latitude'], df.iloc[j]['Longitude']
                distance = haversine(lat1, lon1, lat2, lon2)
                if distance <= max_distance:
                    distances.append((distance, df.iloc[j]['FrameCnt']))
        
        # Sort by distance and select up to max_neighbors closest images
        distances.sort()
        neighbors[df.iloc[i]['FrameCnt']] = [j for _, j in distances[:max_neighbors]]
    
    return neighbors

# Example usage
# Assuming `gps_df` is a DataFrame with 'Latitude' and 'Longitude' columns for each image/frame
neighbors = get_closest_neighbors(gps_df)

gps_df['Neighbors'] = gps_df['FrameCnt'].map(neighbors)

gps_df.to_csv(output_csv, index=False)

print(neighbors)


In [ ]:
import cv2
import os
from tqdm import tqdm  # Import tqdm for the progress bar
import numpy as np

# Paths
video_path = "Dataset/DJI_0763.MOV"  # Path to the input video
txt_folder_path = "Dataset/DJI_0763_detection"  # Path to the folder containing bounding box .txt files
output_video_path = "Dataset/bounding_boxes.mp4"  # Path to save the output video with bounding boxes

# Open the video
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define the codec and create a VideoWriter object to save the output video
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

frame_idx = 0

# Use tqdm for the progress bar
with tqdm(total=total_frames, desc="Processing Video", unit="frame") as pbar:
    # Loop through the video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Exit when no more frames are available

        # Corresponding .txt file for the current frame (assuming sequential filenames)
        txt_file_path = os.path.join(txt_folder_path, f"det_fr_{frame_idx:04d}.txt")

        if os.path.exists(txt_file_path):
            # Read bounding boxes from the .txt file
            with open(txt_file_path, "r") as f:
                lines = f.readlines()

            for line in lines:
                # Split the line into values and parse them
                values = line.strip().split(',')
                # Extract the coordinates for the quadrilateral
                x1, y1, x2, y2, x3, y3, x4, y4 = map(int, values[:8])
                class_name = values[8]  # Object class
                confidence = float(values[9])  # Confidence score

                # Calculate the bounding rectangle (top-left and bottom-right coordinates)
                x_min = min(x1, x2, x3, x4)
                y_min = min(y1, y2, y3, y4)
                x_max = max(x1, x2, x3, x4)
                y_max = max(y1, y2, y3, y4)

                # Draw the rectangle bounding box
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

                # Add the label (class and confidence score)
                label = f"{class_name} {confidence:.2f}"
                cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Write the frame with bounding boxes to the output video
        out.write(frame)

        # Update the progress bar
        pbar.update(1)

        # Increment the frame index
        frame_idx += 1

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video with bounding boxes saved at {output_video_path}")


In [ ]:
import torch
from PIL import Image, ImageDraw
from simple_lama_inpainting import SimpleLama

# Step 1: Initialize SimpleLama
device = 'cuda' if torch.cuda.is_available() else 'cpu'
simple_lama = SimpleLama(device=device)

# Step 2: Set Paths and Parameters
general_path = '/home/oussama/Documents/EPFL/PDS_LUTS/'
image_path = general_path + "/images/250.jpg"
txt_file_path = general_path + "Dataset/DJI_0763_detection/det_fr_0000.txt"
output_image_path = general_path + "0_wo_boxes.jpg"

# Load the image and resize if needed (or keep original resolution for better quality)
image = Image.open(image_path).convert("RGB")
dimensions = (3840, 2160)  # FHD example; adjust based on quality and memory
fhd_image = image.resize(dimensions)
mask = Image.new("L", dimensions, 0)  

# Scaling factors from 4K to FHD
scaling_factor_x = dimensions[0] / 3840
scaling_factor_y = dimensions[1] / 2160

# Offset margin (extend box by these many pixels)
offset = 10

# Draw mask with bounding boxes + offset
draw = ImageDraw.Draw(mask)
with open(txt_file_path, "r") as f:
    lines = f.readlines()

    for line in lines:
        values = list(map(int, line.strip().split(',')[:8]))
        
        # Original coordinates, scaled down
        x1, y1 = int(values[0] * scaling_factor_x), int(values[1] * scaling_factor_y)
        x2, y2 = int(values[2] * scaling_factor_x), int(values[3] * scaling_factor_y)
        x3, y3 = int(values[4] * scaling_factor_x), int(values[5] * scaling_factor_y)
        x4, y4 = int(values[6] * scaling_factor_x), int(values[7] * scaling_factor_y)

        # Adjusted bounding box coordinates with offset
        x_min = min(x1, x2, x3, x4) - offset
        y_min = min(y1, y2, y3, y4) - offset
        x_max = max(x1, x2, x3, x4) + offset
        y_max = max(y1, y2, y3, y4) + offset
        
        # Draw the extended bounding box on the mask
        draw.rectangle([x_min, y_min, x_max, y_max], fill=255)

# mask.show()  # Optional: verify the mask visually

# Perform inpainting with the modified mask
result = simple_lama(fhd_image, mask)
result.save(output_image_path)
result.show()
print(f"Inpainted image saved at {output_image_path}")


In [5]:
import torch
import os
from PIL import Image, ImageDraw
from simple_lama_inpainting import SimpleLama
from tqdm import tqdm
import re

dimensions = (3840, 2160)

# Initialize SimpleLama
device = 'cuda' if torch.cuda.is_available() else 'cpu'
simple_lama = SimpleLama(device=device)

# Set Paths
general_path = '/home/oussama/Documents/EPFL/PDS_LUTS/'
input_folder = os.path.join(general_path, "images_distance")
txt_folder = os.path.join(general_path, "Dataset/DJI_0763_detection")
output_folder = os.path.join(general_path, f"images_updated")

clear_folder(output_folder)

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Dimensions and scaling factors
scaling_factor_x = dimensions[0] / 3840
scaling_factor_y = dimensions[1] / 2160
offset = 10  # Offset margin for bounding boxes

# Regular expression patterns to extract frame numbers
image_pattern = re.compile(r'(\d+)\.jpg$')
txt_pattern = re.compile(r'det_fr_(\d+)\.txt$')

# Create dictionaries for image files and txt files based on frame numbers
image_files = {int(image_pattern.search(f).group(1)): f for f in os.listdir(input_folder) if image_pattern.search(f)}
txt_files = {int(txt_pattern.search(f).group(1)): f for f in os.listdir(txt_folder) if txt_pattern.search(f)}

# Sort frame numbers for sequential processing
frame_numbers = sorted(set(image_files.keys()) & set(txt_files.keys()))

# Loop through each frame with a progress bar
for frame_number in tqdm(frame_numbers, desc="Processing images"):
    image_file = image_files[frame_number]
    txt_file = txt_files[frame_number]

    # Load the image
    image_path = os.path.join(input_folder, image_file)
    image = Image.open(image_path).convert("RGB")
    fhd_image = image.resize(dimensions)
    mask = Image.new("L", dimensions, 0)

    # Load the corresponding txt file
    txt_file_path = os.path.join(txt_folder, txt_file)
    
    # Draw mask with bounding boxes + offset
    draw = ImageDraw.Draw(mask)
    with open(txt_file_path, "r") as f:
        lines = f.readlines()

        for line in lines:
            values = list(map(int, line.strip().split(',')[:8]))

            # Scale and offset bounding box coordinates
            x1, y1 = int(values[0] * scaling_factor_x), int(values[1] * scaling_factor_y)
            x2, y2 = int(values[2] * scaling_factor_x), int(values[3] * scaling_factor_y)
            x3, y3 = int(values[4] * scaling_factor_x), int(values[5] * scaling_factor_y)
            x4, y4 = int(values[6] * scaling_factor_x), int(values[7] * scaling_factor_y)

            x_min = min(x1, x2, x3, x4) - offset
            y_min = min(y1, y2, y3, y4) - offset
            x_max = max(x1, x2, x3, x4) + offset
            y_max = max(y1, y2, y3, y4) + offset

            draw.rectangle([x_min, y_min, x_max, y_max], fill=255)

    # Perform inpainting
    result = simple_lama(fhd_image, mask)

    # Save the result
    output_image_path = os.path.join(output_folder, image_file)
    result.save(output_image_path)


Processing images: 100%|██████████| 33/33 [00:59<00:00,  1.80s/it]
